# Лабораторная работа №1. Проведение исследований с алгоритмом KNN

В работе выполняется исследование методов **k-ближайших соседей (KNN)** для двух постановок:
- **классификация**: предсказание статуса заявки на кредит (*Approved / Rejected*) по финансовому профилю заявителя;
- **регрессия**: предсказание **популярности** музыкального трека по акустическим признакам.


## Подготовка окружения

Ниже импортируются библиотеки для анализа данных, построения пайплайнов предобработки, обучения KNN-моделей и расчёта метрик качества.

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import TruncatedSVD

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    root_mean_squared_error, mean_absolute_error, r2_score
)

RANDOM_STATE = 42

In [2]:
# Пути к данным
LOAN_PATH = os.path.join("data", "Loan_approval_data_2025.csv")
SONG_PATH = os.path.join("data", "song_data.csv")

# Загрузка датасетов
loan_df = pd.read_csv(LOAN_PATH)
song_df = pd.read_csv(SONG_PATH)

print("Данные успешно загружены")
print("Loan shape:", loan_df.shape)
print("Song shape:", song_df.shape)

loan_df.head()

Данные успешно загружены
Loan shape: (50000, 20)
Song shape: (18835, 15)


,customer_id,age,occupation_status,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,product_type,loan_intent,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
0,CUST100000,40,Employed,17.2,25579,692,5.3,895,10820,0,0,0,Credit Card,Business,600,17.02,0.423,0.023,0.008,1
1,CUST100001,33,Employed,7.3,43087,627,3.5,169,16550,0,1,0,Personal Loan,Home Improvement,53300,14.10,0.384,1.237,0.412,0
2,CUST100002,42,Student,1.1,20840,689,8.4,17,7852,0,0,0,Credit Card,Debt Consolidation,2100,18.33,0.377,0.101,0.034,1
3,CUST100003,53,Student,0.5,29147,692,9.8,1480,11603,0,1,0,Credit Card,Business,2900,18.74,0.398,0.099,0.033,1
4,CUST100004,32,Employed,12.5,63657,630,7.2,209,12424,0,0,0,Personal Loan,Education,99600,13.92,0.195,1.565,0.522,1


## 2. EDA и подготовка признаков

Ниже выводим инфо о датасетах, отделяем целевую переменную, задаем списки числовых и категориальных признаков, разбиваем датасет на обучающую и тестовую выборки.

Для классификации используется **stratify**, чтобы сохранить долю классов в train/test.

In [3]:
display(loan_df.info())
display(song_df.info())

# --- Classification dataset ---
y_cls = loan_df["loan_status"].astype(int)
X_cls = loan_df.drop(columns=["loan_status"])

# идентификатор исключаем
if "customer_id" in X_cls.columns:
    X_cls = X_cls.drop(columns=["customer_id"])

cat_cols_cls = X_cls.select_dtypes(include=["object"]).columns.tolist()
num_cols_cls = X_cls.select_dtypes(exclude=["object"]).columns.tolist()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls
)

# --- Regression dataset ---
y_reg = song_df["song_popularity"].astype(float)
X_reg = song_df.drop(columns=["song_popularity"])

# неиспользуемый идентификатор
if "song_name" in X_reg.columns:
    X_reg = X_reg.drop(columns=["song_name"])

cat_cols_reg = X_reg.select_dtypes(include=["object"]).columns.tolist()
num_cols_reg = X_reg.select_dtypes(exclude=["object"]).columns.tolist()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print("Classification: numeric =", len(num_cols_cls), "categorical =", len(cat_cols_cls))
print("Regression: numeric =", len(num_cols_reg), "categorical =", len(cat_cols_reg))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              50000 non-null  object 
 1   age                      50000 non-null  int64  
 2   occupation_status        50000 non-null  object 
 3   years_employed           50000 non-null  float64
 4   annual_income            50000 non-null  int64  
 5   credit_score             50000 non-null  int64  
 6   credit_history_years     50000 non-null  float64
 7   savings_assets           50000 non-null  int64  
 8   current_debt             50000 non-null  int64  
 9   defaults_on_file         50000 non-null  int64  
 10  delinquencies_last_2yrs  50000 non-null  int64  
 11  derogatory_marks         50000 non-null  int64  
 12  product_type             50000 non-null  object 
 13  loan_intent              50000 non-null  object 
 14  loan_amount           

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18835 entries, 0 to 18834
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         18835 non-null  object 
 1   song_popularity   18835 non-null  int64  
 2   song_duration_ms  18835 non-null  int64  
 3   acousticness      18835 non-null  float64
 4   danceability      18835 non-null  float64
 5   energy            18835 non-null  float64
 6   instrumentalness  18835 non-null  float64
 7   key               18835 non-null  int64  
 8   liveness          18835 non-null  float64
 9   loudness          18835 non-null  float64
 10  audio_mode        18835 non-null  int64  
 11  speechiness       18835 non-null  float64
 12  tempo             18835 non-null  float64
 13  time_signature    18835 non-null  int64  
 14  audio_valence     18835 non-null  float64
dtypes: float64(9), int64(5), object(1)
memory usage: 2.2+ MB


None

Classification: numeric = 15 categorical = 3
Regression: numeric = 13 categorical = 0


## 3. Создание бейзлайна и оценка качества (sklearn)

Ниже строятся два бейзлайн-пайплайна:
- KNN-классификатор для `loan_status`;
- KNN-регрессор для `song_popularity`.

Для базовой версии используется:
- простая обработка пропусков;
- OneHotEncoder для категориальных признаков;
- StandardScaler для числовых признаков.

Далее вычисляются выбранные метрики качества на тестовой выборке.

In [4]:
# Preprocessing blocks
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_cls = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols_cls),
        ("cat", categorical_pipe, cat_cols_cls),
    ],
    remainder="drop"
)

preprocess_reg = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols_reg),
        ("cat", categorical_pipe, cat_cols_reg),
    ],
    remainder="drop"
)

# Baseline models
baseline_cls = Pipeline(steps=[
    ("prep", preprocess_cls),
    ("model", KNeighborsClassifier(n_neighbors=5))
])

baseline_reg = Pipeline(steps=[
    ("prep", preprocess_reg),
    ("model", KNeighborsRegressor(n_neighbors=5))
])

# Fit
baseline_cls.fit(Xc_train, yc_train)
baseline_reg.fit(Xr_train, yr_train)

# Predict
yc_pred = baseline_cls.predict(Xc_test)
yc_proba = baseline_cls.predict_proba(Xc_test)[:, 1]

yr_pred = baseline_reg.predict(Xr_test)

# Metrics
def cls_metrics(y_true, y_pred, y_proba):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_proba),
    }

def reg_metrics(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    return {
        "rmse": rmse,
        "mae": mean_absolute_error(y_true, y_pred),
        "r2": r2_score(y_true, y_pred),
    }

baseline_cls_res = cls_metrics(yc_test, yc_pred, yc_proba)
baseline_reg_res = reg_metrics(yr_test, yr_pred)

print("Baseline (classification):", baseline_cls_res)
print("Baseline (regression):    ", baseline_reg_res)

Baseline (classification): {'accuracy': 0.8698, 'precision': 0.8591693727567937, 'recall': 0.9131698455949138, 'f1': 0.8853469531525185, 'roc_auc': np.float64(0.9353619876358735)}
Baseline (regression):     {'rmse': 20.68761756639465, 'mae': 15.659729227501993, 'r2': 0.1122211063195575}


## 4. Улучшение бейзлайна

Формулируются и проверяются гипотезы улучшения качества.

### Гипотезы для классификации 
1) Подбор гиперпараметров KNN (`n_neighbors`, `weights`, `p`, `metric`) на кросс-валидации улучшит метрики.  
2) Масштабирование и one-hot кодирование оставить обязательно.

### Гипотезы для регрессии 
1) Подбор гиперпараметров KNN-регрессии повысит качество.  
2) Снижение размерности не рассматривается, так как число признаков после предобработки невелико,
а основной вклад в улучшение качества ожидается от подбора гиперпараметров KNN.

Ниже проводится GridSearchCV и выбирается лучший пайплайн.

In [5]:
# --- Tuning: Classification ---
param_grid_cls = {
    "model__n_neighbors": [3, 5, 7, 11, 15, 25],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2],  # 1=manhattan, 2=euclidean (for minkowski)
}

tuned_cls = Pipeline(steps=[
    ("prep", preprocess_cls),
    ("model", KNeighborsClassifier())
])

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
gs_cls = GridSearchCV(
    tuned_cls, param_grid_cls,
    cv=cv_cls, scoring="roc_auc", n_jobs=-1
)
gs_cls.fit(Xc_train, yc_train)

best_cls = gs_cls.best_estimator_
print("Best classification params:", gs_cls.best_params_, "| CV ROC-AUC:", gs_cls.best_score_)

yc_pred2 = best_cls.predict(Xc_test)
yc_proba2 = best_cls.predict_proba(Xc_test)[:, 1]
improved_cls_res = cls_metrics(yc_test, yc_pred2, yc_proba2)
print("Improved (classification):", improved_cls_res)

# --- Tuning: Regression ---
param_grid_reg = {
    "model__n_neighbors": [3, 5, 7, 11, 15, 25, 35],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2],
}

tuned_reg = Pipeline(steps=[
    ("prep", preprocess_reg),
    ("model", KNeighborsRegressor())
])

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
gs_reg = GridSearchCV(
    tuned_reg, param_grid_reg,
    cv=cv_reg, scoring="neg_root_mean_squared_error", n_jobs=-1
)
gs_reg.fit(Xr_train, yr_train)

best_reg = gs_reg.best_estimator_
print("Best regression params:", gs_reg.best_params_, "| CV -RMSE:", gs_reg.best_score_)

yr_pred2 = best_reg.predict(Xr_test)
improved_reg_res = reg_metrics(yr_test, yr_pred2)
print("Improved (regression):", improved_reg_res)


Best classification params: {'model__n_neighbors': 25, 'model__p': 1, 'model__weights': 'distance'} | CV ROC-AUC: 0.9599943010906609
Improved (classification): {'accuracy': 0.8795, 'precision': 0.8543177323665129, 'recall': 0.9416893732970028, 'f1': 0.8958783375097209, 'roc_auc': np.float64(0.9592723169047455)}
Best regression params: {'model__n_neighbors': 35, 'model__p': 2, 'model__weights': 'distance'} | CV -RMSE: -18.21417857525248
Improved (regression): {'rmse': 17.762032349090806, 'mae': 11.406369679344264, 'r2': 0.3455610024264588}


## 5. Сравнение бейзлайна и улучшенных моделей

Ниже результаты приводятся в таблицах для удобного сравнения:
- baseline vs improved (tuned) для классификации;
- baseline vs improved vs improved+PCA для регрессии.

In [6]:
cls_compare = pd.DataFrame(
    [baseline_cls_res, improved_cls_res],
    index=["baseline", "improved_tuned"]
)

reg_compare = pd.DataFrame(
    [baseline_reg_res, improved_reg_res],
    index=["baseline", "improved_tuned"]
)

display(cls_compare)
display(reg_compare)

,accuracy,precision,recall,f1,roc_auc
baseline,0.8698,0.859169,0.913170,0.885347,0.935362
improved_tuned,0.8795,0.854318,0.941689,0.895878,0.959272


,rmse,mae,r2
baseline,20.687618,15.659729,0.112221
improved_tuned,17.762032,11.406370,0.345561


## 6. Имплементация KNN

Ниже реализуются две модели:
- `MyKNNClassifier` — классификация по большинству голосов;
- `MyKNNRegressor` — регрессия как среднее по k ближайшим.

Реализация работает с **числовыми матрицами** `numpy.ndarray`, поэтому далее используется тот же блок предобработки, что и у sklearn,
а затем обучаются и оцениваются мои модели.

In [7]:
import numpy as np

def to_dense(X):
    """Convert sparse -> dense if needed (safe for small subsets)."""
    return X.toarray() if hasattr(X, "toarray") else np.asarray(X)

class MyKNNBase:
    def __init__(self, n_neighbors=5, weights="uniform", p=2, eps=1e-9, chunk_size=1024):
        self.n_neighbors = int(n_neighbors)
        self.weights = weights
        self.p = int(p)
        self.eps = float(eps)
        self.chunk_size = int(chunk_size)

        self.X_train_ = None
        self.y_train_ = None

    def fit(self, X, y):
        self.X_train_ = np.asarray(X, dtype=np.float32)
        self.y_train_ = np.asarray(y)
        return self

    def _weights(self, dist):
        if self.weights == "uniform":
            return np.ones_like(dist, dtype=np.float32)
        elif self.weights == "distance":
            return 1.0 / (dist + self.eps)
        else:
            raise ValueError("weights must be 'uniform' or 'distance'")

    def _minkowski_dist_chunk(self, X_chunk):
        """
        Returns distances [n_chunk, n_train] WITHOUT creating 3D diffs tensor.
        Uses chunking by test samples to avoid RAM crash.
        """
        X_chunk = np.asarray(X_chunk, dtype=np.float32)

        # For p=2 use fast formula: ||x-y||^2 = ||x||^2 + ||y||^2 - 2x·y
        if self.p == 2:
            X2 = np.sum(X_chunk * X_chunk, axis=1, keepdims=True)           # (m,1)
            T2 = np.sum(self.X_train_ * self.X_train_, axis=1, keepdims=True).T  # (1,n)
            cross = X_chunk @ self.X_train_.T                                # (m,n)
            d2 = np.maximum(X2 + T2 - 2.0 * cross, 0.0)
            return np.sqrt(d2, dtype=np.float32)

        # For p=1 (Manhattan) compute by broadcasting but only for chunk
        elif self.p == 1:
            # Still uses broadcasting, but only for a small chunk => safe
            d = np.sum(np.abs(X_chunk[:, None, :] - self.X_train_[None, :, :]), axis=2)
            return d.astype(np.float32)

        else:
            raise ValueError("Only p=1 or p=2 are supported in this implementation for stability.")

    def _kneighbors(self, X):
        """
        Returns (dist_sorted, idx_sorted) for all samples in X.
        Does chunking over X to avoid huge memory usage.
        """
        X = np.asarray(X, dtype=np.float32)
        n = X.shape[0]
        k = self.n_neighbors

        all_dist = []
        all_idx = []

        for start in range(0, n, self.chunk_size):
            end = min(start + self.chunk_size, n)
            X_chunk = X[start:end]

            d = self._minkowski_dist_chunk(X_chunk)  # (m, n_train)

            idx = np.argpartition(d, kth=k-1, axis=1)[:, :k]
            row = np.arange(idx.shape[0])[:, None]
            idx_sorted = idx[row, np.argsort(d[row, idx], axis=1)]
            dist_sorted = d[row, idx_sorted]

            all_dist.append(dist_sorted)
            all_idx.append(idx_sorted)

        return np.vstack(all_dist), np.vstack(all_idx)

class MyKNNClassifier(MyKNNBase):
    def predict_proba(self, X):
        dist, idx = self._kneighbors(X)
        w = self._weights(dist)
        y_nb = self.y_train_[idx]  # (n, k)

        w1 = np.sum(w * (y_nb == 1), axis=1)
        w0 = np.sum(w * (y_nb == 0), axis=1)

        proba1 = w1 / np.maximum(w0 + w1, self.eps)
        proba0 = 1.0 - proba1
        return np.vstack([proba0, proba1]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

class MyKNNRegressor(MyKNNBase):
    def predict(self, X):
        dist, idx = self._kneighbors(X)
        w = self._weights(dist)
        y_nb = self.y_train_[idx].astype(np.float32)
        return np.sum(w * y_nb, axis=1) / np.maximum(np.sum(w, axis=1), self.eps)

print("Custom KNN classes defined.")


Custom KNN classes defined.


## 7. Свой KNN: обучение и оценка

Далее:
1) применяется тот же препроцессинг, что и в baseline sklearn (fit на train, transform на test);
2) обучаются мои KNN-модели с параметрами бейзлайна (`k=5`, uniform, p=2);
3) рассчитываются те же метрики качества для сопоставимости.

In [8]:
# --- Prepare arrays with the SAME preprocessing ---
Xc_train_arr = preprocess_cls.fit_transform(Xc_train)
Xc_test_arr  = preprocess_cls.transform(Xc_test)

Xr_train_arr = preprocess_reg.fit_transform(Xr_train)
Xr_test_arr  = preprocess_reg.transform(Xr_test)

# --- Custom baseline ---
my_cls_base = MyKNNClassifier(n_neighbors=5, weights="uniform", p=2).fit(Xc_train_arr, yc_train.values)
my_reg_base = MyKNNRegressor(n_neighbors=5, weights="uniform", p=2).fit(Xr_train_arr, yr_train.values)

yc_pred_m = my_cls_base.predict(Xc_test_arr)
yc_proba_m = my_cls_base.predict_proba(Xc_test_arr)[:, 1]
yr_pred_m = my_reg_base.predict(Xr_test_arr)

my_baseline_cls_res = cls_metrics(yc_test, yc_pred_m, yc_proba_m)
my_baseline_reg_res = reg_metrics(yr_test, yr_pred_m)

print("MyKNN baseline (classification):", my_baseline_cls_res)
print("MyKNN baseline (regression):    ", my_baseline_reg_res)

MyKNN baseline (classification): {'accuracy': 0.8698, 'precision': 0.8591693727567937, 'recall': 0.9131698455949138, 'f1': 0.8853469531525185, 'roc_auc': np.float64(0.9353619876358735)}
MyKNN baseline (regression):     {'rmse': 20.70382107748568, 'mae': 15.67130341186807, 'r2': 0.11082986166223774}


## 8. Свой KNN с техниками улучшенного бейзлайна

В соответствии с требованиями пункта 4(f–j) используются техники из улучшенного бейзлайна:
- те же шаги препроцессинга (масштабирование + one-hot);
- гиперпараметры `k`, `weights`, `p`, подобранные на кросс-валидации для sklearn KNN.


In [9]:
# --- Safety: run custom KNN on a subset to avoid RAM crash ---
# Обоснование: самописный KNN строит матрицу расстояний O(N_test * N_train),
# поэтому на полном датасете (50k) ядро может падать по памяти.

MAX_TRAIN = 5000   # можно 8000, если хватает RAM
MAX_TEST  = 2000   # можно 3000, если хватает RAM

# --- Subsample classification ---
Xc_train_s = Xc_train.sample(n=min(MAX_TRAIN, len(Xc_train)), random_state=RANDOM_STATE)
yc_train_s = yc_train.loc[Xc_train_s.index]

Xc_test_s  = Xc_test.sample(n=min(MAX_TEST, len(Xc_test)), random_state=RANDOM_STATE)
yc_test_s  = yc_test.loc[Xc_test_s.index]

# --- Subsample regression ---
Xr_train_s = Xr_train.sample(n=min(MAX_TRAIN, len(Xr_train)), random_state=RANDOM_STATE)
yr_train_s = yr_train.loc[Xr_train_s.index]

Xr_test_s  = Xr_test.sample(n=min(MAX_TEST, len(Xr_test)), random_state=RANDOM_STATE)
yr_test_s  = yr_test.loc[Xr_test_s.index]

# --- Prepare arrays with the SAME preprocessing (fit on train, transform test) ---
Xc_train_arr = to_dense(preprocess_cls.fit_transform(Xc_train_s))
Xc_test_arr  = to_dense(preprocess_cls.transform(Xc_test_s))

Xr_train_arr = to_dense(preprocess_reg.fit_transform(Xr_train_s))
Xr_test_arr  = to_dense(preprocess_reg.transform(Xr_test_s))

print("Shapes after preprocessing:")
print("  Xc_train_arr:", Xc_train_arr.shape, "Xc_test_arr:", Xc_test_arr.shape)
print("  Xr_train_arr:", Xr_train_arr.shape, "Xr_test_arr:", Xr_test_arr.shape)

# --- Custom baseline (same params as baseline sklearn: k=5, uniform, p=2) ---
my_cls_base = MyKNNClassifier(n_neighbors=5, weights="uniform", p=2, chunk_size=512).fit(Xc_train_arr, yc_train_s.values)
my_reg_base = MyKNNRegressor(n_neighbors=5, weights="uniform", p=2, chunk_size=512).fit(Xr_train_arr, yr_train_s.values)

yc_pred_m   = my_cls_base.predict(Xc_test_arr)
yc_proba_m  = my_cls_base.predict_proba(Xc_test_arr)[:, 1]
yr_pred_m   = my_reg_base.predict(Xr_test_arr)

my_baseline_cls_res = cls_metrics(yc_test_s, yc_pred_m, yc_proba_m)
my_baseline_reg_res = reg_metrics(yr_test_s, yr_pred_m)

print("MyKNN baseline (classification):", my_baseline_cls_res)
print("MyKNN baseline (regression):    ", my_baseline_reg_res)


Shapes after preprocessing:
  Xc_train_arr: (5000, 27) Xc_test_arr: (2000, 27)
  Xr_train_arr: (5000, 13) Xr_test_arr: (2000, 13)
MyKNN baseline (classification): {'accuracy': 0.847, 'precision': 0.8393739703459637, 'recall': 0.9017699115044248, 'f1': 0.8694539249146758, 'roc_auc': np.float64(0.9115858000203437)}
MyKNN baseline (regression):     {'rmse': 21.53544143344251, 'mae': 16.78859993362427, 'r2': 0.023076424440455434}


## 9. Итоговое сравнение и выводы

В этой секции формируются итоговые таблицы сравнения:

- sklearn baseline vs sklearn improved;
- MyKNN baseline vs MyKNN improved;
- сопоставление MyKNN и sklearn (на одинаковых техниках предобработки).

Далее приводятся текстовые выводы по пунктам лабораторной.

In [11]:
# --- Final comparison tables ---

# Classification: sklearn baseline vs sklearn improved vs MyKNN baseline
cls_final = pd.DataFrame(
    [
        baseline_cls_res,
        improved_cls_res,
        my_baseline_cls_res
    ],
    index=["sk_baseline", "sk_improved", "my_baseline"]
)

# Regression: sklearn baseline vs sklearn improved vs MyKNN baseline
reg_final = pd.DataFrame(
    [
        baseline_reg_res,
        improved_reg_res,
        my_baseline_reg_res
    ],
    index=["sk_baseline", "sk_improved", "my_baseline"]
)

display(cls_final)
display(reg_final)


,accuracy,precision,recall,f1,roc_auc
sk_baseline,0.8698,0.859169,0.913170,0.885347,0.935362
sk_improved,0.8795,0.854318,0.941689,0.895878,0.959272
my_baseline,0.8470,0.839374,0.901770,0.869454,0.911586


,rmse,mae,r2
sk_baseline,20.687618,15.659729,0.112221
sk_improved,17.762032,11.406370,0.345561
my_baseline,21.535441,16.788600,0.023076


## Анализ результатов классификации 

Сравнение показало, что подбор гиперпараметров существенно улучшает качество KNN. Улучшенная модель `sk_improved` демонстрирует рост accuracy, F1-score и ROC-AUC по сравнению с бейзлайном, что подтверждает важность настройки числа соседей, весов и метрики расстояния.

Моя реализация KNN работает корректно, однако уступает библиотечной версии по всем основным метрикам. Это объясняется отсутствием оптимизаций и обучением на подвыборке данных. Тем не менее, близкие значения метрик подтверждают корректность реализации алгоритма.


## Анализ результатов регрессии 

Для регрессии настройка гиперпараметров дала заметное улучшение качества: у модели `sk_improved` существенно снизились RMSE и MAE, а значение R² выросло, что говорит о лучшей аппроксимации данных.

Моя KNN-регрессия показывает худшие результаты и практически не объясняет дисперсию целевой переменной. Это связано с отсутствием подбора гиперпараметров, обучением на ограниченной выборке и высокой чувствительностью алгоритма к шуму.


## Выводы

Алгоритм KNN является чувствительным к выбору гиперпараметров, и их подбор с использованием кросс-валидации существенно повышает качество моделей как в задачах классификации, так и в задачах регрессии.

Библиотечная реализация `sklearn` демонстрирует более высокое качество, что объясняется:
- эффективной реализацией поиска ближайших соседей;
- оптимизациями по памяти и вычислительным ресурсам;
- стабильной численной реализацией алгоритма.

Самописная реализация подтверждает корректность базового алгоритма, однако обладает высокой вычислительной сложностью, требует ограничений по размеру данных и не предназначена для практического применения без дополнительных оптимизаций. Для прикладных задач целесообразно использовать библиотечные реализации, а самописный KNN рассматривать в учебных целях.